<a href="https://colab.research.google.com/github/Pornwalaifoxall/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก google drive โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 'sub_district', 'zone' จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

In [13]:
import pandas as pd
import numpy as np

In [107]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
Data_House = pd.read_csv('/content/drive/MyDrive/BSC_DPDM24_DATA/Data House - บ้านมือ2.csv')
Data_House.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# ตรวจสอบข้อมูลเบื้องต้น
print(Data_House.head())  # ตัวอย่างข้อมูล
print(Data_House.info())  # ตรวจสอบโครงสร้างข้อมูล
print(Data_House.describe())  # สถิติพื้นฐาน
print("\nจำนวน Missing Values ในแต่ละคอลัมน์:")
print(Data_House.isnull().sum())

   id                                   name_th  \
0   1                   บ้านเดี่ยว ชนบท ขอนแก่น   
1   4                 บ้านเดี่ยว น้ำพอง ขอนแก่น   
2  16              บ้านเดี่ยว มัญจาคีรี ขอนแก่น   
3  18     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   
4  19  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม   

                                    name_en    project_name  \
0                   บ้านเดี่ยว ชนบท ขอนแก่น     บ้านวังเวิน   
1                 บ้านเดี่ยว น้ำพอง ขอนแก่น  บ้านหนองบัวบาน   
2              บ้านเดี่ยว มัญจาคีรี ขอนแก่น             NaN   
3     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   ชลลดา ขอนแก่น   
4  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม        เบสท์โฮม   

                                 company province      district sub_district  \
0                                    NaN  ขอนแก่น          ชนบท  ศรีบุญเรือง   
1                                    NaN  ขอนแก่น        น้ำพอง      ทรายมูล   
2                                    NaN  ขอนแก่น     มัญจาคีรี      กุดเค

# ตรวจสอบและแก้ไข Missing Value

In [17]:
# เติม Missing Values
numeric_columns = ['number_of_rai', 'number_of_ngan', 'number_of_va', 'usable_area', 'sell_price']
categorical_columns = ['renovate', 'direction', 'furniture', 'district', 'sub_district', 'zone']

In [18]:
# เติมคอลัมน์ตัวเลขด้วยค่าเฉลี่ย
for col in numeric_columns:
    Data_House[col].fillna(Data_House[col].mean(), inplace=True)

<ipython-input-18-c7fab621f93b>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Data_House[col].fillna(Data_House[col].mean(), inplace=True)


In [19]:
# เติมคอลัมน์ข้อความด้วย "ไม่ระบุ"
for col in categorical_columns:
    Data_House[col].fillna("ไม่ระบุ", inplace=True)

<ipython-input-19-717d869b1b05>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Data_House[col].fillna("ไม่ระบุ", inplace=True)


In [20]:
# ตรวจสอบ Missing Values หลังการแก้ไข
print("\nMissing Values หลังการแก้ไข:")
print(Data_House.isnull().sum())


Missing Values หลังการแก้ไข:
id                       0
name_th                  1
name_en                  5
project_name          1117
company               1308
province                 0
district                 0
sub_district             0
property_type            0
home improvement       199
renovate                 0
public utility        1638
land type             1643
land wide(m)          1600
land long(m)          1609
topsoil               1637
corner                1627
zip_code               475
zone                     0
alley                 1401
street                 995
latitude                22
longitude               22
number_of_rai            0
number_of_ngan           0
number_of_va             0
construct_year        1656
model                 1656
sell_price               0
number_of_unit        1638
number_of_story        549
number_of_bedroom       27
number_of_bathroom       0
number_of_parking      215
usable_area              0
central_cost           62

In [109]:
Data_House.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# กำจัด outlier

In [22]:
# ตรวจสอบ Outliers ด้วย IQR
def remove_outliers(Data_House, column):
    Q1 = Data_House[column].quantile(0.25)
    Q3 = Data_House[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return Data_House[(Data_House[column] >= lower_bound) & (Data_House[column] <= upper_bound)]

In [23]:
# กำจัด Outliers ใน sell_price และ usable_area
Data_House = remove_outliers(Data_House, 'sell_price')
Data_House = remove_outliers(Data_House, 'usable_area')

In [24]:
print("\nข้อมูลหลังจากกำจัด Outliers:")
print(Data_House.describe())


ข้อมูลหลังจากกำจัด Outliers:
       land wide(m)  land long(m)      zip_code     latitude    longitude  \
count     47.000000     40.000000  1.096000e+03  1482.000000  1482.000000   
mean      14.502128     18.782500  7.339304e+06    16.860384   103.076089   
std        8.392193     12.871831  1.707935e+08     6.411894     6.199150   
min        4.000000      6.000000  4.000000e+00    15.690442   101.851062   
25%        8.550000     10.350000  4.000000e+04    16.397971   102.729271   
50%       12.500000     16.000000  4.000000e+04    16.434853   102.800725   
75%       16.500000     20.000000  4.014000e+04    16.488446   102.842191   
max       46.000000     63.000000  4.000040e+09   111.000000   222.000000   

       number_of_rai  number_of_ngan  number_of_va  construct_year  model  \
count    1503.000000     1503.000000   1503.000000             0.0    0.0   
mean        0.291417        0.345975     60.024478             NaN    NaN   
std         3.030071        2.531045     57.9

In [25]:
Data_House.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,สุรภาธานี,NaN,ขอนแก่น,เมืองขอนแก่น,ศิลา,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# การทำ normalization
เพื่อปรับขนาดของข้อมูลให้อยู่ช่วงที่มีค่าเท่ากัน

In [26]:
from sklearn.preprocessing import MinMaxScaler

In [27]:
# เลือกคอลลัมน์สำหรับ Normalization
columns_to_normalize = ['number_of_rai', 'number_of_ngan', 'number_of_va', 'usable_area', 'sell_price']

In [28]:
# ใช้ MinMaxScaler
scaler = MinMaxScaler()
Data_House[columns_to_normalize] = scaler.fit_transform(Data_House[columns_to_normalize])

In [29]:
print("\nข้อมูลหลังจาก Normalization:")
print(Data_House.describe())


ข้อมูลหลังจาก Normalization:
       land wide(m)  land long(m)      zip_code     latitude    longitude  \
count     47.000000     40.000000  1.096000e+03  1482.000000  1482.000000   
mean      14.502128     18.782500  7.339304e+06    16.860384   103.076089   
std        8.392193     12.871831  1.707935e+08     6.411894     6.199150   
min        4.000000      6.000000  4.000000e+00    15.690442   101.851062   
25%        8.550000     10.350000  4.000000e+04    16.397971   102.729271   
50%       12.500000     16.000000  4.000000e+04    16.434853   102.800725   
75%       16.500000     20.000000  4.014000e+04    16.488446   102.842191   
max       46.000000     63.000000  4.000040e+09   111.000000   222.000000   

       number_of_rai  number_of_ngan  number_of_va  construct_year  model  \
count    1503.000000     1503.000000   1503.000000             0.0    0.0   
mean        0.004350        0.003642      0.076367             NaN    NaN   
std         0.045225        0.026643      0.0

In [30]:
Data_House.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,สุรภาธานี,NaN,ขอนแก่น,เมืองขอนแก่น,ศิลา,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ทำ one-hot-encoder
โดยการแปลงคอลลัมน์ที่เป็นข้อความ เช่น direction furniture

In [31]:
# ทำ One-Hot Encoding
Data_House = pd.get_dummies(Data_House, columns=['direction', 'furniture', 'district', 'sub_district', 'zone'], drop_first=True)

print("\nข้อมูลหลังจากทำ One-Hot Encoding:")
print(Data_House.head())



ข้อมูลหลังจากทำ One-Hot Encoding:
   id                                    name_th  \
0   1                    บ้านเดี่ยว ชนบท ขอนแก่น   
1   4                  บ้านเดี่ยว น้ำพอง ขอนแก่น   
3  18      บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   
4  19   บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม   
5  20  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี   

                                     name_en    project_name  \
0                    บ้านเดี่ยว ชนบท ขอนแก่น     บ้านวังเวิน   
1                  บ้านเดี่ยว น้ำพอง ขอนแก่น  บ้านหนองบัวบาน   
3      บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   ชลลดา ขอนแก่น   
4   บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม        เบสท์โฮม   
5  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี       สุรภาธานี   

                                 company province property_type  \
0                                    NaN  ขอนแก่น    บ้านเดี่ยว   
1                                    NaN  ขอนแก่น    บ้านเดี่ยว   
3  บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน)  ขอนแก่น    บ้านเดี่ยว   

In [32]:
Data_House.head()

,id,name_th,name_en,project_name,company,province,property_type,home improvement,renovate,public utility,...,zone_มิตรภาพ-บ้านสำราญ,zone_ศิลา-โกทา-หนองไผ่-บ้านดอน,zone_สามเหลี่ยม-กังสดาล-ม.ขอนแก่น,zone_เหล่านาดี-บ้านสะอาด-บ้านโจด,zone_แดงใหญ่,zone_โคกสี,zone_โนนทัน-บึงแก่นนคร,zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,zone_ในเมือง-รื่นรมย์-เทพารักษ์,zone_ไม่ระบุ
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,บ้านเดี่ยว,no,no,NaN,...,False,False,False,False,False,False,False,False,False,True
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,บ้านเดี่ยว,no,no,NaN,...,False,False,False,False,False,False,False,False,False,True
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,บ้านเดี่ยว,no,no,NaN,...,False,False,False,False,False,False,False,False,False,False
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,บ้านเดี่ยว,no,no,NaN,...,False,False,False,False,False,False,False,False,False,False
5,20,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,สุรภาธานี,NaN,ขอนแก่น,บ้านเดี่ยว,no,no,NaN,...,False,True,False,False,False,False,False,False,False,False


In [33]:
#  ตรวจสอบ Missing Value
print("\nFinal Missing Values Check:")
print(Data_House.isnull().sum())


Final Missing Values Check:
id                                          0
name_th                                     1
name_en                                     5
project_name                             1004
company                                  1182
                                         ... 
zone_โคกสี                                  0
zone_โนนทัน-บึงแก่นนคร                      0
zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง       0
zone_ในเมือง-รื่นรมย์-เทพารักษ์             0
zone_ไม่ระบุ                                0
Length: 232, dtype: int64


# จัดการค่า Missing ใน column project_name , name_th , name_en, company และ public utility


In [34]:
# เติม Missing Values ในคอลัมน์ข้อความ
text_columns = ['name_th', 'name_en', 'project_name', 'company']

for col in text_columns:
    Data_House[col].fillna("ไม่ระบุ", inplace=True)


<ipython-input-34-b713284a2a2f>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Data_House[col].fillna("ไม่ระบุ", inplace=True)


In [35]:
Data_House.head()

,id,name_th,name_en,project_name,company,province,property_type,home improvement,renovate,public utility,...,zone_มิตรภาพ-บ้านสำราญ,zone_ศิลา-โกทา-หนองไผ่-บ้านดอน,zone_สามเหลี่ยม-กังสดาล-ม.ขอนแก่น,zone_เหล่านาดี-บ้านสะอาด-บ้านโจด,zone_แดงใหญ่,zone_โคกสี,zone_โนนทัน-บึงแก่นนคร,zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,zone_ในเมือง-รื่นรมย์-เทพารักษ์,zone_ไม่ระบุ
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,ไม่ระบุ,ขอนแก่น,บ้านเดี่ยว,no,no,NaN,...,False,False,False,False,False,False,False,False,False,True
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,ไม่ระบุ,ขอนแก่น,บ้านเดี่ยว,no,no,NaN,...,False,False,False,False,False,False,False,False,False,True
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,บ้านเดี่ยว,no,no,NaN,...,False,False,False,False,False,False,False,False,False,False
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,บ้านเดี่ยว,no,no,NaN,...,False,False,False,False,False,False,False,False,False,False
5,20,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,สุรภาธานี,ไม่ระบุ,ขอนแก่น,บ้านเดี่ยว,no,no,NaN,...,False,True,False,False,False,False,False,False,False,False


In [36]:
# ตรวจสอบค่าขาดหายในคอลัมน์ 'public utility'
print(Data_House['public utility'].isnull().sum())

1489


In [37]:
# ตรวจสอบค่าผิดปกติในคอลัมน์ 'public utility'
print(Data_House['public utility'].unique())

[nan 'ไม่ระบุ' 'yes']


In [38]:
# เติมค่าขาดหายด้วย 'unknown' หรือค่าอื่นๆ ที่เหมาะสม
Data_House['public utility'].fillna('unknown', inplace=True)

<ipython-input-38-72d01d96efc5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Data_House['public utility'].fillna('unknown', inplace=True)


In [39]:
# ถ้ามีค่าว่างจริง (""), แทนที่ด้วย 'unknown'
Data_House['public utility'].replace('', 'unknown', inplace=True)

In [40]:
# ตรวจสอบ Missing Values อีกครั้ง
print("\nตรวจสอบ Missing Values หลังการแก้ไขเพิ่มเติม:")
print(Data_House.isnull().sum())


ตรวจสอบ Missing Values หลังการแก้ไขเพิ่มเติม:
id                                       0
name_th                                  0
name_en                                  0
project_name                             0
company                                  0
                                        ..
zone_โคกสี                               0
zone_โนนทัน-บึงแก่นนคร                   0
zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง    0
zone_ในเมือง-รื่นรมย์-เทพารักษ์          0
zone_ไม่ระบุ                             0
Length: 232, dtype: int64


In [41]:
Data_House.head()

,id,name_th,name_en,project_name,company,province,property_type,home improvement,renovate,public utility,...,zone_มิตรภาพ-บ้านสำราญ,zone_ศิลา-โกทา-หนองไผ่-บ้านดอน,zone_สามเหลี่ยม-กังสดาล-ม.ขอนแก่น,zone_เหล่านาดี-บ้านสะอาด-บ้านโจด,zone_แดงใหญ่,zone_โคกสี,zone_โนนทัน-บึงแก่นนคร,zone_โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,zone_ในเมือง-รื่นรมย์-เทพารักษ์,zone_ไม่ระบุ
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,ไม่ระบุ,ขอนแก่น,บ้านเดี่ยว,no,no,unknown,...,False,False,False,False,False,False,False,False,False,True
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,ไม่ระบุ,ขอนแก่น,บ้านเดี่ยว,no,no,unknown,...,False,False,False,False,False,False,False,False,False,True
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,บ้านเดี่ยว,no,no,unknown,...,False,False,False,False,False,False,False,False,False,False
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,บ้านเดี่ยว,no,no,unknown,...,False,False,False,False,False,False,False,False,False,False
5,20,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,สุรภาธานี,ไม่ระบุ,ขอนแก่น,บ้านเดี่ยว,no,no,unknown,...,False,True,False,False,False,False,False,False,False,False


* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

In [48]:
import math

In [44]:
import json

# ฟังก์ชันโหลดข้อมูลจากไฟล์ JSON
def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

In [47]:
# โหลดข้อมูลจากไฟล์ JSON
seven_eleven_data = load_data('/content/drive/MyDrive/BSC_DPDM24_DATA/7-eleven.json')  # ข้อมูล 7-11
school_data = load_data('/content/drive/MyDrive/BSC_DPDM24_DATA/school.json')  # ข้อมูลโรงเรียน

#### ฟังก์ชันคำนวณระยะทางระหว่างจุดสองจุด

In [53]:
# ฟังก์ชันคำนวณระยะทางระหว่างสองจุด
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)  # คำนวณต่างของ latitude
    delta_lambda = math.radians(lon2 - lon1)  # คำนวณต่างของ longitude

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c  # Distance in kilometers
    return distance

#### ฟังก์ชันตรวจสอบบ้านที่อยู่ในรัศมี 1 กิโลเมตรจาก 7-eleven หรือโรงเรียน

In [54]:
def check_nearby_places(house_data, places_data, radius_km=1):
    nearby_places = []
    for house in house_data:
        house_lat = house['latitude']
        house_lon = house['longitude']
        for place in places_data:
            place_lat = place['latitude']
            place_lon = place['longitude']
            distance = haversine(house_lat, house_lon, place_lat, place_lon)
            if distance <= radius_km:
                nearby_places.append(house['id'])
                break  # ถ้าเจอที่ใกล้เคียงแล้ว ไม่ต้องเช็คที่ต่อไป
    return nearby_places

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

In [63]:
# โหลดข้อมูลจากไฟล์
file_path = '/content/drive/MyDrive/BSC_DPDM24_DATA/คำตอบนักประเมิน(ID) - บ้าน.csv'  # แก้ไขตามเส้นทางไฟล์ของคุณ
Data_evaluator = pd.read_csv(file_path)

In [64]:
Data_evaluator.head()

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,500,000","5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,700,000","โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,200,000","5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,400,000","ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,"6,050,000","5,000,000","5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [67]:
# กรองข้อมูลเฉพาะนักประเมินที่มี ID=01
Data_evaluator_1 = Data_evaluator[Data_evaluator['ID'] == '1']
Data_evaluator_1

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
74,1,https://nayoo.co/khonkaen/posts/1,1,บ้านเดี่ยว,มั่นใจ,"390,000","600,000","3,000-30,000 บาท/ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
75,2,https://nayoo.co/khonkaen/posts/1,1,บ้านเดี่ยว,มั่นใจ,"390,000","1,300,000","ราคาเสนอขาย 1,000 - 3,000 บาท / ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน


In [75]:
# เปลี่ยนชื่อ columns ประเมินราคาซื้อ-ขาย และ ให้คะแนนทรัพย์
Data_evaluator_1 = Data_evaluator_1.rename(columns={'ประเมินราคาซื้อ-ขาย': 'estimator', 'คะแนนทรัพย์': 'score'})

In [76]:
Data_evaluator_1.head()

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,estimator,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,estimator,score
74,1,https://nayoo.co/khonkaen/posts/1,1,บ้านเดี่ยว,มั่นใจ,"390,000","600,000","3,000-30,000 บาท/ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,"390,000",4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
75,2,https://nayoo.co/khonkaen/posts/1,1,บ้านเดี่ยว,มั่นใจ,"390,000","1,300,000","ราคาเสนอขาย 1,000 - 3,000 บาท / ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,"390,000",4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน


# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

In [77]:
# โหลดข้อมูลจากไฟล์
file_path = '/content/drive/MyDrive/BSC_DPDM24_DATA/คำตอบนักประเมิน(ID) - บ้าน.csv'  # แก้ไขตามเส้นทางไฟล์ของคุณ
Data_evaluator = pd.read_csv(file_path)

In [78]:
Data_evaluator

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,500,000","5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,700,000","โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,200,000","5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,400,000","ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,"6,050,000","5,000,000","5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,"965,000","550,000","ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,"1,000,000","1,000,000","ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,"10,873,000","10,000,000","ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,"1,000,000","1,200,000",ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [113]:
# คำนวณราคาซื้อขายบ้าน (sell price) เฉลี่ยตามแต่ละ zone
avg_sell_price_per_zone = Data_House.groupby('zone')['sell_price'].mean()
avg_sell_price_per_zone

,sell_price
zone,
กลางเมือง-เมืองเก่า-กุดกว้าง,4.457461e+06
ดอนบม-บายพาส-บ้านผือ-พระลับ,3.293429e+06
ทุ่งสร้าง-ศรีจันทร์-ราชมงคล,5.351464e+06
ท่าพระ-หนองบัวดีหมี,2.764172e+06
บึงเนียม,4.587833e+06
บ้านทุ่ม-บ้านม่วง,2.474188e+06
บ้านเป็ด-โคกฟันโปง,3.179203e+06
ม.ภาค-บ้านกอก-บึงหนองโคตร,4.605049e+06
มะลิวัลย์-ร.8-สนามบิน,3.853152e+06


In [114]:
# หาค่าที่สูงที่สุด
max_zone = avg_sell_price_per_zone.idxmax()
max_avg_price = avg_sell_price_per_zone.max()

In [115]:
# แสดงผลลัพธ์
print(f"\nZone ที่มีราคาซื้อขายบ้านเฉลี่ยสูงที่สุดคือ {max_zone} โดยมีราคาซื้อขายเฉลี่ย {max_avg_price:.2f}")


Zone ที่มีราคาซื้อขายบ้านเฉลี่ยสูงที่สุดคือ ในเมือง-รื่นรมย์-เทพารักษ์ โดยมีราคาซื้อขายเฉลี่ย 7201666.67


3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [119]:
# เปลี่ยนชื่อคอลัมน์ 'id' เป็น 'ID'
Data_House = Data_House.rename(columns={'id': 'ID'})

In [120]:
Data_House.head()

,ID,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# รวมข้อมูลจากตาราง Data_House และ evaluator_data โดยใช้คอลัมน์ 'ID'
merged_data = pd.merge(Data_evaluator, Data_House, on='ID', how='inner')

In [125]:
merged_data.head()

,ID ผู้ประเมิน,post id,ID,property_type_x,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,500,000","5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,700,000","โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,200,000","5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,400,000","ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,"6,050,000","5,000,000","5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
